In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torch.optim.lr_scheduler import MultiStepLR
import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule
from pytorch_lightning.utilities import rank_zero_info
from torchmetrics.classification.accuracy import Accuracy 

from collections import OrderedDict

import numpy as np
import os

import sys
sys.path.append(os.path.join(os.getcwd(), "../../scripts/"))

from dataset import HDF5Dataset

In [2]:
pl.seed_everything(42)

Global seed set to 42


42

In [3]:
PATH_TO_TRAIN = "/deep/group/aihc-bootcamp-fall2021/lymphoma/processed/data_splits/train.hdf5"
PATH_TO_VAL = "/deep/group/aihc-bootcamp-fall2021/lymphoma/processed/data_splits/val.hdf5"
PATH_TO_TEST = "/deep/group/aihc-bootcamp-fall2021/lymphoma/processed/data_splits/test.hdf5"

In [4]:
# Datasets
train_dataset = HDF5Dataset(PATH_TO_TRAIN)
val_dataset = HDF5Dataset(PATH_TO_VAL)
test_dataset = HDF5Dataset(PATH_TO_TEST)

train_loader = DataLoader(train_dataset, batch_size=1, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=1, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=4)


/deep/u/xiaoliy2/anaconda3/envs/lym/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
######### Fine-tune Model ##########
# Model adapted from net.py as the template model to load pretrained weights
class TripletNet_Finetune(pl.LightningModule):

    def __init__(self):
        super(TripletNet_Finetune, self).__init__()

        # set the model
        model = models.resnet18(pretrained=False)
        model.fc = torch.nn.Sequential()
        self.model = model
        self.fc = nn.Sequential(nn.Linear(512*2, 512),
                                 nn.ReLU(True), nn.Linear(512, 256))

    def forward(self, i):

        E1 = self.model(i)
        E2 = self.model(i)
        E3 = self.model(i)

        # Pairwise concatenation of features
        E12 = torch.cat((E1, E2), dim=1)
        E23 = torch.cat((E2, E3), dim=1)
        E13 = torch.cat((E1, E3), dim=1)

        f12 = self.fc(E12)
        f23 = self.fc(E23)
        f13 = self.fc(E13)

        features = torch.cat((f12, f23, f13), dim=1)

        return features


In [6]:
PATH_TO_PRETRAINED = '/deep/group/aihc-bootcamp-fall2021/lymphoma/models/Camelyon16_pretrained_model.pt'

In [7]:
class SupervisedBaseline(pl.LightningModule):
    def __init__(self, 
                 num_classes, 
                 batch_size: int, 
                 lr: float, 
                 num_workers: int, 
                 finetune: bool = False):
        
        super().__init__()
        self.batch_size = batch_size
        self.lr = lr
        self.num_workers = num_workers
        
        # Load pre-trained network:
        model = TripletNet_Finetune()

        state_dict = torch.load(PATH_TO_PRETRAINED) ## TODO: change this to pytorch lightning

        # create new OrderedDict that does not contain `module`
        new_state_dict = OrderedDict()
        for k, v in state_dict['model'].items():
            name = k[7:]  # remove `module.`
            new_state_dict[name] = v
            
        # load pretrained weights onto TripletNet_Finetune model
        model.load_state_dict(new_state_dict)
        
        # if we finetune - only train the classifier, as opposed to e2e - freeze the network
        if finetune:
            for name, param in enumerate(model.named_parameters()):
                param = param[1]
                param.requires_grad=False
                
        self.feature_extractor = model
        
        # set the linear classifier
        # use the classifier setup in the paper
        self.classifier = nn.Sequential(nn.Linear(256*3, num_classes))
        
        self.criterion = nn.CrossEntropyLoss()
        
        # use separate metric instance for train, val and test step
        # to ensure a proper reduction over the epoch
        self.train_accuracy = Accuracy()
        self.val_accuracy = Accuracy()
        self.test_accuracy = Accuracy()
        
        # ensures params passed to LightningModule will be saved to ckpt
        # allows to access params with 'self.hparams' attribute
        self.save_hyperparameters()
        
    def forward(self, x):
        # Forward step
        x = self.feature_extractor(x).flatten(1)   # representations
        x = self.classifier(x)                     # classifications
        return x
    
    def aggregate(self, y_hats):
        # TODO: confirm argmax/max
        return torch.max(y_hats, dim=0)[0].unsqueeze(0)
        
    def infer(self, bag, y):
        y_hats = []
        for x in bag:
            y_hats.append(self(x).squeeze())
            
        y_hat = self.aggregate(torch.stack(y_hats, dim=0))
        
        return y_hat

    def configure_optimizers(self):
        # only train parameters that are not frozen
        parameters = self.parameters()
        trainable_parameters = list(filter(lambda p: p.requires_grad, parameters))
        
        optimizer = torch.optim.Adam(trainable_parameters, lr=self.lr)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        bag, y = batch
        y_hat = self.infer(bag, y)
        loss = self.criterion(y_hat, y)
        acc = self.train_accuracy(y_hat, y)
    
        # log train metrics
        self.log("train/loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train/acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        return {"loss": loss, "preds": y_hat.detach(), "targets": y.detach()}
    
    def validation_step(self, batch, batch_idx):
        bag, y = batch
        y_hat = self.infer(bag, y)
        loss = self.criterion(y_hat, y)
        acc = self.val_accuracy(y_hat, y)
    
        # log validation metrics
        self.log("val/loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val/acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        return {"loss": loss, "preds": y_hat.detach(), "targets": y.detach()}
    
    def test_step(self, batch, batch_idx):
        bag, y = batch
        y_hat = self.infer(bag, y)
        loss = self.criterion(y_hat, y)
        acc = self.test_accuracy(y_hat, y)
        
        # log test metrics
        self.log("test/loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test/acc", acc, on_step=False, on_epoch=True, prog_bar=True)

        return {"loss": loss, "preds": y_hat.detach(), "targets": y.detach()}

In [12]:
model = SupervisedBaseline(num_classes=9, batch_size=1, lr=1e-4, num_workers=1, finetune=True)

In [13]:
# training
trainer = pl.Trainer(
    gpus=-1, num_nodes=1, num_processes=8,
    precision=16, limit_train_batches=0.5, accelerator="dp",
    max_epochs=1
)
trainer.fit(model, train_loader, val_loader)

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Set SLURM handle signals.

  | Name              | Type                | Params
----------------------------------------------------------
0 | feature_extractor | TripletNet_Finetune | 11.8 M
1 | classifier        | Sequential          | 6.9 K 
2 | criterion         | CrossEntropyLoss    | 0     
3 | train_accuracy    | Accuracy            | 0     
4 | val_accuracy      | Accuracy            | 0     
5 | test_accuracy     | Accuracy            | 0     
----------------------------------------------------------
6.9 K     Trainable params
11.8 M    Non-trainable params
11.8 M    Total params
47.358    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/deep/u/xiaoliy2/anaconda3/envs/lym/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


KeyboardInterrupt: 

In [11]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/acc': 0.43728458881378174, 'test/loss': 2.0847790241241455}
--------------------------------------------------------------------------------


[{'test/loss': 2.0847790241241455, 'test/acc': 0.43728458881378174}]

In [12]:
trainer.save_checkpoint("../../models/test.ckpt")